# create Json for each survery

0) geo tiff aws ARN
1) polygon bounds
2) survey date      
3) species types
4) species metrics
5) species ID file ARN

In [61]:

import json
import httpx
from shapely.geometry import Polygon
import os
import geopandas as gpd


species_shape_file = '/Users/matthewtoberman/Dropbox/TRITONIA/CIVTECH/PORTAL/species_locations/22_Alcyionium/22_Alcyionium.shp'





data = gpd.GeoDataFrame.from_file(species_shape_file)

data_trans = data.to_crs(27700)
data_trans.crs
poly_areas = list(data_trans.geometry.area)
species_area = sum(poly_areas)



os.system('ogr2ogr -f geoJSON' + ' ' + species_shape_file[:-3] + 'json' + ' ' + species_shape_file )

with open(species_shape_file[:-3] + 'json' , 'r') as f:
    species_polys = json.load(f)


COG_arn = "https://tiletesting.s3.eu-west-2.amazonaws.com/COG_ARD_BAY_REEF_22_cropped_1cm.tif"



json_out_path = "/Users/matthewtoberman/Dropbox/TRITONIA/CIVTECH/PORTAL/meta_data/ard_bay_2022.json"


titiler_endpoint= "https://os8ci3nx02.execute-api.eu-west-2.amazonaws.com"  # titiler running on AWS Lamda


def whole_geotiff_polygon(url):
    bounded_data = httpx.get(
    f"{titiler_endpoint}/cog/bounds",
    params = {
        "url": url,
        }
    ).json()
    entire_geotiff_poly = Polygon([[bounded_data['bounds'][0],bounded_data['bounds'][1]],
                      [bounded_data['bounds'][2],bounded_data['bounds'][1]],
                      [bounded_data['bounds'][2],bounded_data['bounds'][3]],
                      [bounded_data['bounds'][0],bounded_data['bounds'][3]]])
    return entire_geotiff_poly
 
def get_centre_geotiff(url):
    bounded_data = httpx.get(
    f"{titiler_endpoint}/cog/bounds",
    params = {
        "url": url,
        }
    ).json()
    center_lon = (bounded_data['bounds'][2]+bounded_data['bounds'][0])/2
    center_lat = (bounded_data['bounds'][1]+bounded_data['bounds'][3])/2
    centre_coords = [center_lon,center_lat]
    return centre_coords



# Data to be written
dictionary = {
    "survey_dash_id": "ard_bay_2022",
    "survey_long_name": "Ardmucknish Bay 2022",
    "COG_ARN" : COG_arn,
    "COG_species_filename" : "/Users/matthewtoberman/Dropbox/TRITONIA/CIVTECH/PORTAL/species_locations/Alcyonium_2022.shp" , 
    "survey_date": '2022',
    "species_name": "Alcyonium digitatum",
    "marker_type":'alcyonium_digitatum_icon_red',
    "number_of_individuals" : 'N/A',
    'species_m2':species_area,
    "centre_coords" :list(reversed(get_centre_geotiff(COG_arn))),
    "species_polygons": species_polys
    
}
 
# Serializing json
json_object = json.dumps(dictionary, indent=4)
 
# Writing to sample.json
with open(json_out_path, "w") as outfile:
    outfile.write(json_object)



In [58]:

import json
import httpx
from shapely.geometry import Polygon
import os
import geopandas as gpd


species_shape_file = '/Users/matthewtoberman/Dropbox/TRITONIA/CIVTECH/PORTAL/species_locations/23_alcyionium/23_Alcyionium.shp'

data = gpd.GeoDataFrame.from_file(species_shape_file)

data_trans = data.to_crs(27700)
data_trans.crs
poly_areas = list(data_trans.geometry.area)
species_area = sum(poly_areas)



os.system('ogr2ogr -f geoJSON' + ' ' + species_shape_file[:-3] + 'json' + ' ' + species_shape_file )

with open(species_shape_file[:-3] + 'json' , 'r') as f:
    species_polys = json.load(f)


COG_arn = "https://tiletesting.s3.eu-west-2.amazonaws.com/COG_ARD_BAY_REEF_23_cropped_1cm.tif"



json_out_path = "/Users/matthewtoberman/Dropbox/TRITONIA/CIVTECH/PORTAL/meta_data/ard_bay_2023.json"


titiler_endpoint= "https://os8ci3nx02.execute-api.eu-west-2.amazonaws.com"  # titiler running on AWS Lamda


def whole_geotiff_polygon(url):
    bounded_data = httpx.get(
    f"{titiler_endpoint}/cog/bounds",
    params = {
        "url": url,
        }
    ).json()
    entire_geotiff_poly = Polygon([[bounded_data['bounds'][0],bounded_data['bounds'][1]],
                      [bounded_data['bounds'][2],bounded_data['bounds'][1]],
                      [bounded_data['bounds'][2],bounded_data['bounds'][3]],
                      [bounded_data['bounds'][0],bounded_data['bounds'][3]]])
    return entire_geotiff_poly
 
def get_centre_geotiff(url):
    bounded_data = httpx.get(
    f"{titiler_endpoint}/cog/bounds",
    params = {
        "url": url,
        }
    ).json()
    center_lon = (bounded_data['bounds'][2]+bounded_data['bounds'][0])/2
    center_lat = (bounded_data['bounds'][1]+bounded_data['bounds'][3])/2
    centre_coords = [center_lon,center_lat]
    return centre_coords



# Data to be written
dictionary = {
    "survey_dash_id": "ard_bay_2023",
    "survey_long_name": "Ardmucknish Bay 2023",
    "COG_ARN" : COG_arn,
    "COG_species_filename" : "/Users/matthewtoberman/Dropbox/TRITONIA/CIVTECH/PORTAL/species_locations/Alcyonium_2023.shp" , 
    "survey_date": '2023',
    "species_name": "Alcyonium digitatum",
    "marker_type":'alcyonium_digitatum_icon_red',
    "number_of_individuals" : 'N/A',
    'species_m2':species_area,
    "centre_coords" :list(reversed(get_centre_geotiff(COG_arn))),
    "species_polygons": species_polys
    
}
 
# Serializing json
json_object = json.dumps(dictionary, indent=4)
 
# Writing to sample.json
with open(json_out_path, "w") as outfile:
    outfile.write(json_object)



In [309]:


import json
import httpx
from shapely.geometry import Polygon
import os
import geopandas as gpd


species_shape_file = '/Users/matthewtoberman/Dropbox/TRITONIA/CIVTECH/PORTAL/species_locations/CRERAN_05_04_23_Serpulid_markers/Serpulids.shp'



data = gpd.GeoDataFrame.from_file(species_shape_file)
data_trans = data.to_crs(27700)
data_trans.crs
poly_areas = list(data_trans.geometry.area)
species_area = sum(poly_areas[1:])

print(species_area)



os.system('ogr2ogr -f geoJSON' + ' ' + species_shape_file[:-3] + 'json' + ' ' + species_shape_file )

with open(species_shape_file[:-3] + 'json' , 'r') as f:
    species_polys = json.load(f)

print(len(species_polys['features']))
species_polys['features'] = species_polys['features'][1:]

COG_arn = 'https://tiletesting.s3.eu-west-2.amazonaws.com/COG_GDAL_CrereanSerpOrthoCropped.tif'


json_out_path = "/Users/matthewtoberman/Dropbox/TRITONIA/CIVTECH/PORTAL/meta_data/creran_2023.json"


titiler_endpoint= "https://os8ci3nx02.execute-api.eu-west-2.amazonaws.com"  # titiler running on AWS Lamda


def whole_geotiff_polygon(url):
    bounded_data = httpx.get(
    f"{titiler_endpoint}/cog/bounds",
    params = {
        "url": url,
        }
    ).json()
    entire_geotiff_poly = Polygon([[bounded_data['bounds'][0],bounded_data['bounds'][1]],
                      [bounded_data['bounds'][2],bounded_data['bounds'][1]],
                      [bounded_data['bounds'][2],bounded_data['bounds'][3]],
                      [bounded_data['bounds'][0],bounded_data['bounds'][3]]])
    return entire_geotiff_poly
 
def get_centre_geotiff(url):
    bounded_data = httpx.get(
    f"{titiler_endpoint}/cog/bounds",
    params = {
        "url": url,
        }
    ).json()
    center_lon = (bounded_data['bounds'][2]+bounded_data['bounds'][0])/2
    center_lat = (bounded_data['bounds'][1]+bounded_data['bounds'][3])/2
    centre_coords = [center_lon,center_lat]
    return centre_coords



# Data to be written
dictionary = {
    "survey_dash_id": "creran_2023",
    "survey_long_name": "Creran 2023",
    "COG_ARN" : COG_arn,
    "COG_species_filename" : species_shape_file, 
    "survey_date": '2023',
    "species_name": "Serpula vermicularis",
    "marker_type":'serp_icon_red',
    "number_of_individuals" : len(species_polys['features']),
    'species_m2':species_area,
    "centre_coords" :list(reversed(get_centre_geotiff(COG_arn))),
    "species_polygons": species_polys

    
}
 
# Serializing json
json_object = json.dumps(dictionary, indent=4)
 
# Writing to sample.json
with open(json_out_path, "w") as outfile:
    outfile.write(json_object)


0.6686401849606671
31


In [95]:
import json
import httpx
from shapely.geometry import Polygon
# species_shape_file = '/Users/matthewtoberman/Dropbox/TRITONIA/CIVTECH/PORTAL/species_locations/Aligned_with_CRERAN_SERPULIDS_05_04_23/Serpulid_polygons/Serpulid_polygons.shp'
# data = gpd.GeoDataFrame.from_file(species_shape_file)
# data_trans = data.to_crs(27700)
# data_trans.crs
# poly_areas = list(data_trans.geometry.area)
# species_area = sum(poly_areas[1:])

# os.system('ogr2ogr -f geoJSON' + ' ' + species_shape_file[:-3] + 'json' + ' ' + species_shape_file )

# with open(species_shape_file[:-3] + 'json' , 'r') as f:
#     species_polys = json.load(f)


COG_arn = 'https://tiletesting.s3.eu-west-2.amazonaws.com/COG_CRERAN_15_03_23_T1_1mm_cropped.tif'


json_out_path = "/Users/matthewtoberman/Dropbox/TRITONIA/CIVTECH/PORTAL/meta_data/creran_T1_15_03_2023.json"


titiler_endpoint= "https://os8ci3nx02.execute-api.eu-west-2.amazonaws.com"  # titiler running on AWS Lamda


def whole_geotiff_polygon(url):
    bounded_data = httpx.get(
    f"{titiler_endpoint}/cog/bounds",
    params = {
        "url": url,
        }
    ).json()
    entire_geotiff_poly = Polygon([[bounded_data['bounds'][0],bounded_data['bounds'][1]],
                      [bounded_data['bounds'][2],bounded_data['bounds'][1]],
                      [bounded_data['bounds'][2],bounded_data['bounds'][3]],
                      [bounded_data['bounds'][0],bounded_data['bounds'][3]]])
    return entire_geotiff_poly
 
def get_centre_geotiff(url):
    bounded_data = httpx.get(
    f"{titiler_endpoint}/cog/bounds",
    params = {
        "url": url,
        }
    ).json()
    center_lon = (bounded_data['bounds'][2]+bounded_data['bounds'][0])/2
    center_lat = (bounded_data['bounds'][1]+bounded_data['bounds'][3])/2
    centre_coords = [center_lon,center_lat]
    return centre_coords



# Data to be written
dictionary = {
    "survey_dash_id": "creran_T1_15_03_2023",
    "survey_long_name": "Creran T1 15 March 2023",
    "COG_ARN" : COG_arn,
    "COG_species_filename" : "/Users/matthewtoberman/Dropbox/TRITONIA/CIVTECH/PORTAL/ortho_geotiff/CreranSerpClassified_downsample.tif" , 
    "survey_date": '2023',
    "species_name": "Serpula vermicularis",
    "marker_type":'serp_icon_red',
    "number_of_individuals" : 0,
    'species_m2':0,
    "centre_coords" :list(reversed(get_centre_geotiff(COG_arn))),
    "species_polygons":' '

    
}
 
# Serializing json
json_object = json.dumps(dictionary, indent=4)
 
# Writing to sample.json
with open(json_out_path, "w") as outfile:
    outfile.write(json_object)


In [315]:
import json
import httpx
from shapely.geometry import Polygon
species_shape_file = '/Users/matthewtoberman/Dropbox/TRITONIA/CIVTECH/PORTAL/species_locations/CentralCreagan/CentralCreagan_HorseMussels.shp'


os.system('ogr2ogr -f geoJSON' + ' ' + species_shape_file[:-3] + 'json' + ' ' + species_shape_file )

with open(species_shape_file[:-3] + 'json' , 'r') as f:
    species_polys = json.load(f)

COG_arn = 'https://tiletesting.s3.eu-west-2.amazonaws.com/COG_CentralCreagan_HorseMussels_1cm.tif'


json_out_path = "/Users/matthewtoberman/Dropbox/TRITONIA/CIVTECH/PORTAL/meta_data/CentralCreagan_HorseMussels_2023.json"


titiler_endpoint= "https://os8ci3nx02.execute-api.eu-west-2.amazonaws.com"  # titiler running on AWS Lamda


def whole_geotiff_polygon(url):
    bounded_data = httpx.get(
    f"{titiler_endpoint}/cog/bounds",
    params = {
        "url": url,
        }
    ).json()
    entire_geotiff_poly = Polygon([[bounded_data['bounds'][0],bounded_data['bounds'][1]],
                      [bounded_data['bounds'][2],bounded_data['bounds'][1]],
                      [bounded_data['bounds'][2],bounded_data['bounds'][3]],
                      [bounded_data['bounds'][0],bounded_data['bounds'][3]]])
    return entire_geotiff_poly
 
def get_centre_geotiff(url):
    bounded_data = httpx.get(
    f"{titiler_endpoint}/cog/bounds",
    params = {
        "url": url,
        }
    ).json()
    center_lon = (bounded_data['bounds'][2]+bounded_data['bounds'][0])/2
    center_lat = (bounded_data['bounds'][1]+bounded_data['bounds'][3])/2
    centre_coords = [center_lon,center_lat]
    return centre_coords



# Data to be written
dictionary = {
    "survey_dash_id": "CentralCreagan_HorseMussels_2023",
    "survey_long_name": "Central Creagan HorseMussels March 2023",
    "COG_ARN" : COG_arn,
    "survey_date": '2023',
    "species_name": "Modiolus modiolus",
    "marker_type":'modiolus_modiolus_icon',
    "number_of_individuals" :  len(species_polys['features']),
    'species_m2':'N/A',
    "centre_coords" :list(reversed(get_centre_geotiff(COG_arn))),
     "species_polygons": species_polys

    
}
 
# Serializing json
json_object = json.dumps(dictionary, indent=4)
 
# Writing to sample.json
with open(json_out_path, "w") as outfile:
    outfile.write(json_object)


In [91]:
import json
import httpx
from shapely.geometry import Polygon
species_shape_file = '/Users/matthewtoberman/Dropbox/TRITONIA/CIVTECH/PORTAL/species_locations/CreaganBridge_DIVING_Pam_horse_mussels/CreaganBridge_DIVING_Pam_Ortho_horse_mussels.shp'

os.system('ogr2ogr -f geoJSON' + ' ' + species_shape_file[:-3] + 'json' + ' ' + species_shape_file )

with open(species_shape_file[:-3] + 'json' , 'r') as f:
    species_polys = json.load(f)

COG_arn = 'https://tiletesting.s3.eu-west-2.amazonaws.com/COG_CreaganBridge_DIVING_Pam_Ortho_1cm.tif'


json_out_path = "/Users/matthewtoberman/Dropbox/TRITONIA/CIVTECH/PORTAL/meta_data/Creagan_Bridge_HorseMussels_2023.json"


titiler_endpoint= "https://os8ci3nx02.execute-api.eu-west-2.amazonaws.com"  # titiler running on AWS Lamda


def whole_geotiff_polygon(url):
    bounded_data = httpx.get(
    f"{titiler_endpoint}/cog/bounds",
    params = {
        "url": url,
        }
    ).json()
    entire_geotiff_poly = Polygon([[bounded_data['bounds'][0],bounded_data['bounds'][1]],
                      [bounded_data['bounds'][2],bounded_data['bounds'][1]],
                      [bounded_data['bounds'][2],bounded_data['bounds'][3]],
                      [bounded_data['bounds'][0],bounded_data['bounds'][3]]])
    return entire_geotiff_poly
 
def get_centre_geotiff(url):
    bounded_data = httpx.get(
    f"{titiler_endpoint}/cog/bounds",
    params = {
        "url": url,
        }
    ).json()
    center_lon = (bounded_data['bounds'][2]+bounded_data['bounds'][0])/2
    center_lat = (bounded_data['bounds'][1]+bounded_data['bounds'][3])/2
    centre_coords = [center_lon,center_lat]
    return centre_coords



# Data to be written
dictionary = {
    "survey_dash_id": "Creagan_Bridge_HorseMussels_2023",
    "survey_long_name": "Creagan Bridge HorseMussels March 2023",
    "COG_ARN" : COG_arn,
    "survey_date": '2023',
    "species_name": "Modiolus modiolus",
    "marker_type":'modiolus_modiolus_icon',
    "number_of_individuals" :len(species_polys['features']),
    'species_m2':'N/A',
    "centre_coords" :list(reversed(get_centre_geotiff(COG_arn))),
     "species_polygons": species_polys
}
 
# Serializing json
json_object = json.dumps(dictionary, indent=4)
 
# Writing to sample.json
with open(json_out_path, "w") as outfile:
    outfile.write(json_object)


In [12]:
import json
import httpx
from shapely.geometry import Polygon


COG_arn = 'https://tiletesting.s3.eu-west-2.amazonaws.com/COG_BN19_PORTAL.tif'


json_out_path = "/Users/matthewtoberman/Dropbox/TRITONIA/CIVTECH/PORTAL/meta_data/COG_BN19_PORTAL.json"


titiler_endpoint= "https://os8ci3nx02.execute-api.eu-west-2.amazonaws.com"  # titiler running on AWS Lamda


def whole_geotiff_polygon(url):
    bounded_data = httpx.get(
    f"{titiler_endpoint}/cog/bounds",
    params = {
        "url": url,
        }
    ).json()
    entire_geotiff_poly = Polygon([[bounded_data['bounds'][0],bounded_data['bounds'][1]],
                      [bounded_data['bounds'][2],bounded_data['bounds'][1]],
                      [bounded_data['bounds'][2],bounded_data['bounds'][3]],
                      [bounded_data['bounds'][0],bounded_data['bounds'][3]]])
    return entire_geotiff_poly
 
def get_centre_geotiff(url):
    bounded_data = httpx.get(
    f"{titiler_endpoint}/cog/bounds",
    params = {
        "url": url,
        }
    ).json()
    center_lon = (bounded_data['bounds'][2]+bounded_data['bounds'][0])/2
    center_lat = (bounded_data['bounds'][1]+bounded_data['bounds'][3])/2
    centre_coords = [center_lon,center_lat]
    return centre_coords



# Data to be written
dictionary = {
    "survey_dash_id": "BN19",
    "survey_long_name": "Lagoon Knoll, Peros Banhos 2019",
    "COG_ARN" : COG_arn,
    "COG_species_filename" : " " , 
    "survey_date": '2019',
    "species_name": "Acropora clathrata",
    "marker_type":'table_coral_icon',
    "number_of_individuals" : 0,
    'species_m2':0,
    "centre_coords" :list(reversed(get_centre_geotiff(COG_arn))),
    "species_polygons":' '

    
}
 
# Serializing json
json_object = json.dumps(dictionary, indent=4)
 
# Writing to sample.json
with open(json_out_path, "w") as outfile:
    outfile.write(json_object)


In [14]:
import json
import httpx
from shapely.geometry import Polygon


COG_arn = 'https://tiletesting.s3.eu-west-2.amazonaws.com/COG_HOPE_PORTAL_1.tif'


json_out_path = "/Users/matthewtoberman/Dropbox/TRITONIA/CIVTECH/PORTAL/meta_data/COG_HOPE_1.json"


titiler_endpoint= "https://os8ci3nx02.execute-api.eu-west-2.amazonaws.com"  # titiler running on AWS Lamda


def whole_geotiff_polygon(url):
    bounded_data = httpx.get(
    f"{titiler_endpoint}/cog/bounds",
    params = {
        "url": url,
        }
    ).json()
    entire_geotiff_poly = Polygon([[bounded_data['bounds'][0],bounded_data['bounds'][1]],
                      [bounded_data['bounds'][2],bounded_data['bounds'][1]],
                      [bounded_data['bounds'][2],bounded_data['bounds'][3]],
                      [bounded_data['bounds'][0],bounded_data['bounds'][3]]])
    return entire_geotiff_poly
 
def get_centre_geotiff(url):
    bounded_data = httpx.get(
    f"{titiler_endpoint}/cog/bounds",
    params = {
        "url": url,
        }
    ).json()
    center_lon = (bounded_data['bounds'][2]+bounded_data['bounds'][0])/2
    center_lat = (bounded_data['bounds'][1]+bounded_data['bounds'][3])/2
    centre_coords = [center_lon,center_lat]
    return centre_coords



# Data to be written
dictionary = {
    "survey_dash_id": "HOPE_2023",
    "survey_long_name": "Hope Reef, Salisi' Besar",
    "COG_ARN" : COG_arn,
    "COG_species_filename" : " " , 
    "survey_date": '2023',
    "species_name": "Acropora clathrata",
    "marker_type":'table_coral_icon',
    "number_of_individuals" : 0,
    'species_m2':0,
    "centre_coords" :list(reversed(get_centre_geotiff(COG_arn))),
    "species_polygons":' '

    
}
 
# Serializing json
json_object = json.dumps(dictionary, indent=4)
 
# Writing to sample.json
with open(json_out_path, "w") as outfile:
    outfile.write(json_object)


In [ ]:
import json
json_out_path = "/Users/matthewtoberman/Dropbox/TRITONIA/CIVTECH/PORTAL/meta_data/all_survey_list.json"

# survey json file list
survey_json_paths = ['/Users/matthewtoberman/Dropbox/TRITONIA/CIVTECH/PORTAL/meta_data/ard_bay_2022.json', 
                     '/Users/matthewtoberman/Dropbox/TRITONIA/CIVTECH/PORTAL/meta_data/ard_bay_2023.json', 
                     '/Users/matthewtoberman/Dropbox/TRITONIA/CIVTECH/PORTAL/meta_data/creran_2023.json',
                     '/Users/matthewtoberman/Dropbox/TRITONIA/CIVTECH/PORTAL/meta_data/creran_T1_15_03_2023.json',
                     '/Users/matthewtoberman/Dropbox/TRITONIA/CIVTECH/PORTAL/meta_data/CentralCreagan_HorseMussels_2023.json',
                     '/Users/matthewtoberman/Dropbox/TRITONIA/CIVTECH/PORTAL/meta_data/Creagan_Bridge_HorseMussels_2023.json']

# Data to be written
dictionary = {
    "survey_json_paths": survey_json_paths
}
 
# Serializing json
json_object = json.dumps(dictionary, indent=4)
 
# Writing to sample.json
with open(json_out_path, "w") as outfile:
    outfile.write(json_object)
